# Watson Assistant Bot v1

In [1]:
import json, sys,csv, os, shutil, datetime, tqdm

from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

from __future__ import print_function

## Declarations

### Parameters

In [2]:
path_to_json_file="credentials/ibm-credentials_STT.env"
os.path.isfile(path_to_json_file)
customization_id='298cc13e-f6ec-4532-84a4-5ad0e726e2e6'

### functions

In [3]:
def json_to_dictionary(path_env_file):
    dic={}
    f = open(path_env_file, 'r')
    for line in f:
        tokens=line.split('=')
        dic[tokens[0]]=tokens[1].replace('\n','')
    f.close()
    return dic


In [4]:
def logger(jsonOBJ, path_log_file='default_log_file.json',write_mode='w'):
    #ts=str(datetime.datetime.now())
    #jsonOBJ={"ts":ts,"res":jsonOBJ}
    jsonOBJ = json.dumps(jsonOBJ, indent = 4)
    logWriter = open(path_log_file, write_mode)

    
    logWriter.write(jsonOBJ)
    logWriter.close()

In [5]:
def recognize_audio(path_audio_file='audio-file2.flac',grammar_name=None,
                    word_alternatives_threshold=0.9,
                    keywords_threshold=None,
                    keywords=None
                   ):
    ext_content_type={'flac':'audio/flac', 'wav':'audio/wav','mp3':'audio/mp3'}
    ext=path_audio_file.split('.')[-1]
    if keywords==None and keywords_threshold !=None:
        print(f'Warning: Ignoring keywords_threshold:{keywords_threshold} because keywords is None')
        keywords_threshold=None
    #print (ext, ext_content_type[ext])
    with open(path_audio_file,'rb') as audio_file:
        speech_recognition_results = speech_to_text.recognize(audio=audio_file,
                                                              content_type=ext_content_type[ext],
                                                              word_alternatives_threshold=word_alternatives_threshold,
                                                              keywords=keywords,#['colorado', 'tornado', 'tornadoes'],
                                                              keywords_threshold=keywords_threshold,
                                                              grammar_name=grammar_name,
                                                              model='en-US_BroadbandModel',
                                                              language_customization_id=customization_id,
                                                             ).get_result()
        return speech_recognition_results
    

## test: Speech to Text

In [6]:
#def main():
# env
env_watson_stt=json_to_dictionary(path_to_json_file)

# instantiate
authenticator = IAMAuthenticator(env_watson_stt['SPEECH_TO_TEXT_APIKEY'])
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url(env_watson_stt['SPEECH_TO_TEXT_URL'])

In [7]:
#speech_models = speech_to_text.list_models().get_result()
speech_model = speech_to_text.get_model('en-US_BroadbandModel').get_result()

In [8]:
speech_recognition_results=recognize_audio(path_audio_file='MI.wav')
speech_recognition_results

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'Mike Cordell infection might Cordell infection ',
     'confidence': 0.33}]}]}

In [9]:
speech_recognition_results=recognize_audio(path_audio_file='MI.wav',grammar_name='GrammarSymptoms-xml')
speech_recognition_results

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'Myocardial Infarction ',
     'confidence': 0.85,
     'language_model_source': 'GrammarSymptoms-xml'}],
   'word_alternatives': [{'start_time': 1.89,
     'end_time': 2.69,
     'alternatives': [{'word': 'Infarction', 'confidence': 1.0}]}]},
  {'final': True,
   'alternatives': [{'transcript': 'Myocardial Infarction ',
     'confidence': 0.85,
     'language_model_source': 'GrammarSymptoms-xml'}],
   'word_alternatives': [{'start_time': 3.5,
     'end_time': 4.19,
     'alternatives': [{'word': 'Myocardial', 'confidence': 1.0}]},
    {'start_time': 4.22,
     'end_time': 4.97,
     'alternatives': [{'word': 'Infarction', 'confidence': 1.0}]}]}]}